In [5]:
#include("SVM.jl")
#using .SVM
using JuMP, Ipopt, LinearAlgebra, Random, Distributions
using Gadfly
using IJulia
IJulia.installkernel("Julia nodeps", "--depwarn=no")


# case 1: 1-bit logical control, 1 transition condition, 1 process (done)
# case 2: 2-bit logical control (ind), 1 transition condition (1 proc-dep), 2 processes (ind) (done)
# case 3: 2-bit logical control (ind), 1 transition codnition (1 proc-dep), 2 processes (rel)
# case 4: 2-bit logical control (rel), 1 transition condition (1 proc-dep), 2 processes (ind)
# case 5: 2-bit logical control (rel), 1 transition condition (1 proc-dep), 2 processes (rel)

# case 6: 1-bit logical control, 2 transition conditions, 1 process

# For transition conditions
# h_1: 1 transition condition for 1-bit control,  based on single process each
# h_1_mp: 1 transition condition for 1-bit control, based on multiple processes
# h_2: 2 transition condition for 2-bit control, based on single process each

# y is n_y column vector
# u is n_u column vector
# A is n_y x n_y matrix, dynamics of physical quantities
# B is n_y x n_u matrix, effect exerted by logical control to physical quantities
function f(A, B, y, u)
    d = Normal()
    e = rand(d, size(y)[1])
    A * y + B * u + e
end

function example_f_case_3()
    # 2-bit control (ind), 2 processes (ind)
    A = Diagonal([0.9; 0.85])
    B = Diagonal([4; 5])
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end


function example_f_case_4()
    # 2-bit control (rel), 2 processes (ind)
    A = Diagonal([0.9; 0.85])
    B = [4  1;
         2  5]
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end


function example_f_case_5()
    # 2-bit control (ind), 2 processes (rel)
    A = [0.9  -0.2;
         0.2  0.85]
    B = Diagonal([4; 5])
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end

function example_f_case_6()
    # 2-bit control (rel), 2 processes (rel)
    A = [0.9  -0.2;
         0.2  0.85]
    B = [4  1;
         2  5]
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end

function example_f_case_7()
    # 2-bit control (rel), 2 processes (rel)
    A = [0.9  -0.2;
         0.2  0.85]
    B = [4  1;
         2  5]
    y = [20; 20]
    u = [0; 0]
    C = [1 -0.2; -1 -0.2; 0.2 1;0.2 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end

# For n_u bit control, there are 2n_u step functions, 2 for each bit.
# C is a 2n_u by n_y matrix with ±1 and 0. C * y means relate to each control bit the relevant processes with positive relation (<) or negative relation (>).
# y is the process
# D is a 2n_u vector
# The (2i)-th row of C * y + D denotes constraint for the i-th bit control = 0. (2i-1)-th row denotes =1.
function H(y, u, C, D)
    # s is boolean vector indicating whether the i-th bit control is on or off by its entry 2i and 2i-1
    s = zeros(size(u)[1] * 2)
    for i in 1:size(u)[1]
        s[2*i-1] = 1 - u[i]
        s[2*i] = u[i]
    end

    step_funs = C * y + D .< 0
    u_next = (u + step_funs[s .== 1]) .% 2
    return u_next
end


function trajectory(T, A, B, y₀, u₀, h)
    (y, u) = (y₀, u₀)
    n_y = size(y)[1]
    n_u = size(u)[1]
    Y = zeros(n_y, T)
    U = zeros(n_y, T)
    for t = 1:T
        u = h(y, u)
        y = f(A, B, y, u)
        U[:, t] = u
        Y[:, t] = y
    end
    (Y, U)
end

function trajectory_H(T, A, B, C, D, y_0, u_0)
    h(y, u) = H(y, u, C, D)
    trajectory(T, A, B, y_0, u_0, h)
end



function ols(Y_out, Y_in, U_in)
    @assert size(Y_out)[1] == size(Y_in)[1]
    @assert size(Y_out)[2] == size(Y_in)[2] == size(U_in)[2]

    n_y, n_t = size(Y_in)
    n_u = size(U_in)[1]
    #@show n_y, n_t, n_u

    model = Model(with_optimizer(Ipopt.Optimizer))

    @variable(model, Â[1:n_y, 1:n_y])
    @variable(model, B̂[1:n_y, 1:n_u])
    @objective(model, Min, sum((Y_out - (Â * Y_in + B̂ * U_in)) .^ 2))

    optimize!(model)
    @show value.(Â)
    @show value.(B̂)

    value.(Â), value.(B̂)
end


function format_ols(Y, U)
    Y_in = Y[:, 1:end-1]
    Y_out = Y[:, 2:end]
    U_in = U[:, 2:end]  # u(t) is first updated with y(t-1) and then used to calculate y(t)

    Y_out, Y_in, U_in
end

function learn_control_svm(Y, X）
    # bad: this probably leaves out those rare transition points!
    # train = bitrand(rng, m)
    # test = .!train
    n, m = size(X)
    train = ones(m) .== 1
    test = train

    #λ = 0.01
    #max_passes = 10000
    #@show model = svm(X[:,train], Y[train]; optimizer=SVM.CDDual, max_passes = max_passes, λ = λ)
    @show model = svm(X[:,train], Y[train]; optimizer=SVM.InteriorPoint)

    @show train_cost = cost(model, X[:,train], Y[train])

    @show train_accuracy = accuracy(model, X[:,train], Y[train])
    @show test_accuracy = accuracy(model, X[:,test], Y[test])
    
function example(example_f_case, T)
    A, B, y, u, C, D = example_f_case()
    Y, U = trajectory_H(T, A, B, C, D, y, u) # Generate simulation data of T 
    Y_out, Y_in, U_in = format_ols(Y, U) # Prepare
    ols(Y_out, Y_in, U_in) # Conduct physical part via ordinary least square
    # learn_control_svm(Y, U） # Learn control part via svm
end

function re_trial(example_f_case, TSeq)# A round
    RE = zeros(2, size(TSeq)[1])
    for (j, T) in enumerate(TSeq)
        A, B = example_f_case()  # Given parameters
        Â, B̂ = example(example_f_case, T) # A round with a perticular sample size
#         println("---------Â----------")
#         @show Â
#         println("---------A----------")
#         @show A
#         println("---------Norm A----------")
#         println(norm(A))
        RE[:, j] = [norm(A - Â) / norm(A); norm(B - B̂) / norm(B)]
    end
    RE
end

function re_n(example_f_case, TSeq, n_trials) # 10 round and save all of the results
    RETrials_A = zeros(n_trials, size(TSeq)[1]) # Save all of the result in a matrix, e.g. 10(n_trials) * 1000(Tseq) 
    RETrials_B = copy(RETrials_A)
    for i in 1:n_trials
        RE = re_trial(example_f_case, TSeq) # A round
        RETrials_A[i, :] = RE[1, :]
        RETrials_B[i, :] = RE[2, :]
    end
    RETrials_A, RETrials_B
end


function re_n_plot(example_f_case, TSeq, n_trials) # 10 round and calculate the mean and var
    RETrials_A, RETrials_B = re_n(example_f_case, TSeq, n_trials)

    REMean_A = mean(RETrials_A, dims=1) # Mean of n trials
    REMean_B = mean(RETrials_B, dims=1)

    REVar_A  = var(RETrials_A, dims=1)
    REVar_B  = var(RETrials_B, dims=1)

    @show REMean_A, REMean_B, REVar_A, REVar_B
end

function plots(example_f_case) # Main Function
    TSeq = [10; 100; Int(1e3); Int(1e4); Int(1e5)] # Different sample number
    n_trials = 10
    m_a, m_b, v_a, v_b = re_n_plot(example_f_case, TSeq, n_trials) # Try n trails with different sample number
    [TSeq m_a' v_a' m_b' v_b']
end

plots(example_f_case_3) # Call
    
    

┌ Info: Installing Julia nodeps kernelspec in /Users/jingwenshi/Library/Jupyter/kernels/julia-nodeps-1.6
└ @ IJulia /Users/jingwenshi/.julia/packages/IJulia/e8kqU/deps/kspec.jl:94


LoadError: syntax: invalid character "）" near column 32

In [4]:
function test() # Main Function
    b = 1
    a = [1 2 3 4]
    c = [1 3 5 7 9]
end
test()

1×5 Matrix{Int64}:
 1  3  5  7  9

In [12]:
using JuMP, Ipopt, LinearAlgebra, Random, Distributions
using Gadfly


# case 1: 1-bit logical control, 1 transition condition, 1 process (done)
# case 2: 2-bit logical control (ind), 1 transition condition (1 proc-dep), 2 processes (ind) (done)
# case 3: 2-bit logical control (ind), 1 transition codnition (1 proc-dep), 2 processes (rel)
# case 4: 2-bit logical control (rel), 1 transition condition (1 proc-dep), 2 processes (ind)
# case 5: 2-bit logical control (rel), 1 transition condition (1 proc-dep), 2 processes (rel)

# case 6: 1-bit logical control, 2 transition conditions, 1 process

# For transition conditions
# h_1: 1 transition condition for 1-bit control,  based on single process each
# h_1_mp: 1 transition condition for 1-bit control, based on multiple processes
# h_2: 2 transition condition for 2-bit control, based on single process each

# y is n_y column vector
# u is n_u column vector
# A is n_y x n_y matrix, dynamics of physical quantities
# B is n_y x n_u matrix, effect exerted by logical control to physical quantities
function f(A, B, y, u)
    d = Normal()
    e = rand(d, size(y)[1])
    A * y + B * u + e
end

function example_f_case_3()
    # 2-bit control (ind), 2 processes (ind)
    A = Diagonal([0.9; 0.85])
    B = Diagonal([4; 5])
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end


function example_f_case_4()
    # 2-bit control (rel), 2 processes (ind)
    A = Diagonal([0.9; 0.85])
    B = [4  1;
         2  5]
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end


function example_f_case_5()
    # 2-bit control (ind), 2 processes (rel)
    A = [0.9  -0.2;
         0.2  0.85]
    B = Diagonal([4; 5])
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end

function example_f_case_6()
    # 2-bit control (rel), 2 processes (rel)
    A = [0.9  -0.2;
         0.2  0.85]
    B = [4  1;
         2  5]
    y = [20; 20]
    u = [0; 0]
    C = [1 0; -1 0; 0 1; 0 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end

function example_f_case_7()
    # 2-bit control (rel), 2 processes (rel)
    A = [0.9  -0.2;
         0.2  0.85]
    B = [4  1;
         2  5]
    y = [20; 20]
    u = [0; 0]
    C = [1 -0.2; -1 -0.2; 0.2 1;0.2 -1]
    D = [-20; 25; -20; 25]
    A, B, y, u, C, D
end

# For n_u bit control, there are 2n_u step functions, 2 for each bit.
# C is a 2n_u by n_y matrix with ±1 and 0. C * y means relate to each control bit the relevant processes with positive relation (<) or negative relation (>).
# y is the process
# D is a 2n_u vector
# The (2i)-th row of C * y + D denotes constraint for the i-th bit control = 0. (2i-1)-th row denotes =1.
function H(y, u, C, D)
    # s is boolean vector indicating whether the i-th bit control is on or off by its entry 2i and 2i-1
    s = zeros(size(u)[1] * 2)
    for i in 1:size(u)[1]
        s[2*i-1] = 1 - u[i]
        s[2*i] = u[i]
    end

    step_funs = C * y + D .< 0
    u_next = (u + step_funs[s .== 1]) .% 2
    return u_next
end


# function h_1(y, u)
#     # 1-bit control transition, each with single process dependence
#     u′ = zeros(size(u))

#     # heater: 1st bit
#     b = 1
#     if u[b] == 0
#         if y[b] < 20  # y[b] - 20 < 0
#             u′[b] = 1
#         else
#             u′[b] = u[b]
#         end
#     else
#         # u == 1
#         if y[b] > 25 # 25 - y[b] < 0
#             u′[b] = 0
#         else
#             u′[b] = u[b]
#         end
#     end
#     u′
# end


# function h_2(y, u)
#     # 2-bit control transition, each with singel process dependence
#     u′ = zeros(size(u))

#     # heater: 1st bit
#     b = 1
#     if u[b] == 0
#         if y[b] < 20
#             u′[b] = 1
#         else
#             u′[b] = u[b]
#         end
#     else
#         # u == 1
#         if y[b] > 25
#             u′[b] = 0
#         else
#             u′[b] = u[b]
#         end
#     end

#     # heater: 2nd bit
#     b = 2
#     if u[b] == 0
#         if y[b] < 20
#             u′[b] = 1
#         else
#             u′[b] = u[b]
#         end
#     else
#         # u == 1
#         if y[b] > 25
#             u′[b] = 0
#         else
#             u′[b] = u[b]
#         end
#     end

#     u′
# end


function trajectory(T, A, B, y₀, u₀, h)
    (y, u) = (y₀, u₀)
    n_y = size(y)[1]
    n_u = size(u)[1]
    Y = zeros(n_y, T)
    U = zeros(n_y, T)
    for t = 1:T
        u = h(y, u)
        y = f(A, B, y, u)
        U[:, t] = u
        Y[:, t] = y
    end
    (Y, U)
end

function trajectory_H(T, A, B, C, D, y_0, u_0)
    h(y, u) = H(y, u, C, D)
    trajectory(T, A, B, y_0, u_0, h)
end



function ols(Y_out, Y_in, U_in)
    @assert size(Y_out)[1] == size(Y_in)[1]
    @assert size(Y_out)[2] == size(Y_in)[2] == size(U_in)[2]

    n_y, n_t = size(Y_in)
    n_u = size(U_in)[1]
    #@show n_y, n_t, n_u

    model = Model(with_optimizer(Ipopt.Optimizer))

   @variable(model, Â[1:n_y, 1:n_y])
   @variable(model, B̂[1:n_y, 1:n_u])
   @objective(model, Min, sum((Y_out - (Â * Y_in + B̂ * U_in)) .^ 2))

    optimize!(model)
    @show value.(Â)
    @show value.(B̂)

    value.(Â), value.(B̂)
end


function format_ols(Y, U)
    Y_in = Y[:, 1:end-1]
    Y_out = Y[:, 2:end]
    U_in = U[:, 2:end]  # u(t) is first updated with y(t-1) and then used to calculate y(t)

    Y_out, Y_in, U_in
end

function example(example_f_case, T)
    A, B, y, u, C, D = example_f_case()
    Y, U = trajectory_H(T, A, B, C, D, y, u)
    Y_out, Y_in, U_in = format_ols(Y, U)
    ols(Y_out, Y_in, U_in)
end

function re_trial(example_f_case, TSeq)
    RE = zeros(2, size(TSeq)[1])
    for (j, T) in enumerate(TSeq)
        A, B = example_f_case()
        Â, B̂ = example(example_f_case, T)
        RE[:, j] = [norm(A - Â) / norm(A); norm(B - B̂) / norm(B)]
        # RERel[:, j] = [norm(A - Â) / norm(A); norm(B - B̂) / norm(B)]
    end
    RE
end

function re_n(example_f_case, TSeq, n_trials)
    RETrials_A = zeros(n_trials, size(TSeq)[1])
    RETrials_B = copy(RETrials_A)
    for i in 1:n_trials
        RE = re_trial(example_f_case, TSeq)
        RETrials_A[i, :] = RE[1, :]
        RETrials_B[i, :] = RE[2, :]
    end
    RETrials_A, RETrials_B
end


function re_n_plot(example_f_case, TSeq, n_trials)
    RETrials_A, RETrials_B = re_n(example_f_case, TSeq, n_trials)

    REMean_A = mean(RETrials_A, dims=1)
    REMean_B = mean(RETrials_B, dims=1)

    REVar_A  = var(RETrials_A, dims=1)
    REVar_B  = var(RETrials_B, dims=1)

    @show REMean_A, REMean_B, REVar_A, REVar_B
end

function plots(example_f_case)
    TSeq = [10; 100; Int(1e3); Int(1e4); Int(1e5)]
    n_trials = 10
    m_a, m_b, v_a, v_b = re_n_plot(example_f_case, TSeq, n_trials)
    [TSeq m_a' v_a' m_b' v_b']
end

plots(example_f_case_3)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:        8
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

5×5 Matrix{Float64}:
     10.0  0.282536    0.0476105    0.312561    0.0147611
    100.0  0.0381644   0.000189733  0.0653713   0.000460148
   1000.0  0.0132069   1.96598e-5   0.0204553   0.000103956
  10000.0  0.00329243  2.88937e-6   0.00711658  3.11534e-6
 100000.0  0.00166788  7.10501e-7   0.00187633  2.89315e-7